In [1]:
import os
import getpass
from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

model=ChatMistralAI(model="mistral-large-latest",temperature=1)
embeddings = MistralAIEmbeddings()
print(model.invoke("Tell me a joke"))


c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


content='What do you call fake spaghetti? An Impasta' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 7, 'total_tokens': 20, 'completion_tokens': 13}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'} id='run-2b9ae769-88d6-497b-8bdc-b0ea3f54b49b-0' usage_metadata={'input_tokens': 7, 'output_tokens': 13, 'total_tokens': 20}


In [3]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

chain= model | parser
chain.invoke("tell me a joke about space.")

'What did the astronaut say when he crashed into the moon? "I Apollo-gize."'

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("wings of fire.pdf")
pages=loader.load_and_split()
pages

[Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.7.19.0 (http://www.pdf-tools.com)', 'creator': 'PyPDF', 'creationdate': '', 'author': 'Printers1', 'title': 'Wings of fire by Abdul Kalam', 'moddate': '2016-11-23T05:41:49+00:00', 'source': 'wings of fire.pdf', 'total_pages': 141, 'page': 3, 'page_label': '4'}, page_content='WINGS OF FIREAn AutobiographyAVUL PAKIR JAINULABDEEN ABDUL KALAM hascome to personally represent to many of his countrymen thebest aspects of Indian life. Born in 1931, the son of a littleeducated boatowner in Rameswaram, Tamilnadu, he hadan unparalleled career as a defence scientist, culminatingin the highest civilian award of India, the Bharat Ratna.\nAs chief of the country’s defence research anddevelopment programme, Kalam demonstrated the greatpotential for dynamism and innovation that existed inseemingly moribund research establishments. This is thestory of Kalam’s rise from obscurity and his personal andprofessional struggles, as well as

In [5]:
from langchain import PromptTemplate

custom_instruction = """
You are a helpful AI assistant. Answer the questions based on the provided context.
If you don't know the answer, reply with "I don't know."

Context: {context}
Question: {question}
"""

# Create a single prompt template with the instruction included
prompt = PromptTemplate.from_template(custom_instruction)

chain = prompt | model | parser

print(chain.invoke({"context": "Pune is getting hotter day by day", "question": "is pune cool or hot?"}))



Based on the provided context, Pune is getting hotter. So, it is hot.


In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore=DocArrayInMemorySearch.from_documents(pages,embedding=embeddings)

c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever=vectorstore.as_retriever()
retriever.invoke("Andrew Tate")

[Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.7.19.0 (http://www.pdf-tools.com)', 'creator': 'PyPDF', 'creationdate': '', 'author': 'Printers1', 'title': 'Wings of fire by Abdul Kalam', 'moddate': '2016-11-23T05:41:49+00:00', 'source': 'wings of fire.pdf', 'total_pages': 141, 'page': 47, 'page_label': '48'}, page_content='unforeseen costs. “You name the thing and I will get it foryou, but do not ask for time,” he said. At times, I oftenlaughed at his impatience, and read for him these linesfrom T.S. Eliot’s Hollow Men:Between the conceptionAnd the creationBetween the emotionAnd the responseFalls the Shadow.Defence R&D at that time was heavily dependent onimported equipment. Virtually nothing indigenous wasavailable. Together, we made a long shopping list anddrew up an import plan. But this made me unhappy—wasthere no remedy or alternative? Was this nation doomed tolive with screwdriver technology? Could a poor country likeIndia afford this kind of development?

In [11]:
from operator import itemgetter

chain=(
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    |parser
)

chain.invoke({"question":"who is Andrew Tate?"})

"I don't know."